In [2]:
import carla
import random

In [3]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [12]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

print(location, rotation)

# Set the spectator to a position
# spectator.set_transform(carla.Transform())
# spectator.set_transform(carla.Transform(carla.Location(x=-25.698477, y=-2.615946, z=14.969325),carla.Rotation(pitch=-28.920618, yaw=135.692627, roll=0.000037)))


Location(x=-62.010975, y=1.288139, z=17.589510) Rotation(pitch=-48.045647, yaw=53.329460, roll=0.000444)


In [13]:
spectator.set_transform(carla.Transform(carla.Location(x=-62.010975, y=1.288139, z=17.589510),carla.Rotation(pitch=-48.045647, yaw=53.329460, roll=0.000444)))


Add Traffic

In [ ]:
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')

In [ ]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library

for i in range(0,200):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [ ]:
# spawn one car:
# car_bp = world.get_blueprint_library().find('*vehicle*')

ego_vehicle = world.spawn_actor(vehicle_blueprints)

In [ ]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

---------- RGB camera ----------

In [ ]:
camera_trans = carla.Transform(carla.Location(z=0.1))

camera_bp= world.get_blueprint_library().find('sensor.camera.rgb')

rgb_camera = world.spawn_actor(camera_bp, camera_trans, attach_to=spectator)

In [ ]:
rgb_camera.listen(lambda image: image.save_to_disk('rgb_output2/%06d.png' % image.frame))

In [ ]:
rgb_camera.stop()

---------- DVS camera ----------

In [ ]:
camera_trans = carla.Transform(carla.Location(z=0.1))

camera_bp= world.get_blueprint_library().find('sensor.camera.dvs')

dvs_camera = world.spawn_actor(camera_bp, camera_trans, attach_to=spectator)

In [ ]:
import csv
import numpy as np
import time
import cv2

In [ ]:
def dvs_callback(data, writer, dvs_output_path): # store in csv file
    print("length = ",len(data))

    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
    with open(dvs_output_path, mode="a",  newline='') as file:
        writer = csv.writer(file)
        # dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        #     ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
        for event in dvs_events:
            writer.writerow(event)
        file.close()
    

In [ ]:
def dvs_callback_img(data): #store in image
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
    # dvs_img = np.zero((data.raw_data.height, data.raw_data.width, 3), dtype=np.uint8)
    # dvs_img[dsv_events[:]['y'],dsv_events[:]['x'],dsv_events[:]['pol']*2] = 255

    dvs_img = data.to_image()
    print(dvs_img)
    # dvs_img.save_to_disk('output/%06d.png' % dvs_img.frame)
    
    

In [ ]:
dvs_output_path = "dvs_output.csv"
with open(dvs_output_path, mode="w",  newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['x', 'y', 't', 'pol'])
    file.close()

In [ ]:
dvs_camera.stop()

In [ ]:
dvs_camera.listen(lambda DVSEventArray: dvs_callback_img(DVSEventArray))
# time.sleep(3)
# dvs_camera.stop()

In [ ]:
dvs_camera.listen(lambda DVSEventArray: dvs_callback(DVSEventArray, writer, dvs_output_path))
time.sleep(3)
dvs_camera.stop()

In [ ]:

dvs_output_path = "dvs_output.csv"

with open(dvs_output_path, mode="r",  newline='') as file:
    first = 0
    second = 0
    reader = csv.reader(file)
    for row in reader:
        second = int(row[2])
        if first > second:
            print("ERROR!!!")
            print(first, second)
        first = int(row[2])
    file.close()